## CARGA Y VALIDACIÓN

In [1]:
import pandas as pd

df = pd.read_csv("ventas.csv")
df["fecha"] = pd.to_datetime(df["fecha"])

df.head()

,fecha,sede,cliente_id,servicio,monto,metodo_pago,canal,vendedor
0,2024-01-03,Nezahualcoyotl,C001,Consulta Inicial,500,Efectivo,Presencial,Ana Lopez
1,2024-01-03,Balbuena,C002,Plan 10 Sesiones,4500,Transferencia,Presencial,Carlos Ruiz
2,2024-01-04,Chalco,C003,Sesion Individual,600,Tarjeta,Presencial,Laura Perez
3,2024-01-04,Nezahualcoyotl,C004,Consulta Inicial,500,Efectivo,Digital,Ana Lopez
4,2024-01-05,Balbuena,C005,Plan 15 Sesiones,6500,Transferencia,Presencial,Carlos Ruiz


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   fecha        17 non-null     datetime64[ns]
 1   sede         17 non-null     object        
 2   cliente_id   17 non-null     object        
 3   servicio     17 non-null     object        
 4   monto        17 non-null     int64         
 5   metodo_pago  17 non-null     object        
 6   canal        17 non-null     object        
 7   vendedor     17 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 1.2+ KB


## KPI 1: VENTAS POR SEDE

In [3]:
ventas_por_sede = (
    df.groupby("sede")["monto"]
    .sum()
    .sort_values(ascending=False)
)

ventas_por_sede

sede
Balbuena          19800
Nezahualcoyotl    12600
Chalco             6200
Coapa              6100
Name: monto, dtype: int64

## Hallazgo 1: Concentración de ingresos por sede

El análisis muestra que una parte relevante de los ingresos se concentra en un número reducido de sedes, lo que sugiere diferencias claras en desempeño comercial.

## KPI 2: TICKET PROMEDIO POR SEDE

In [4]:
ticket_promedio = (
    df.groupby("sede")["monto"]
    .mean()
    .round(2)
    .sort_values(ascending=False)
)

ticket_promedio

sede
Balbuena          4950.0
Nezahualcoyotl    2520.0
Chalco            1550.0
Coapa             1525.0
Name: monto, dtype: float64

## Hallazgo 2: Diferencias en ticket promedio

Existen diferencias relevantes en el ticket promedio entre sedes, lo que indica estrategias comerciales o mix de servicios distintos.

## TABLA EJECUTIVA POR SEDE

In [5]:
resumen_sede = (
    df.groupby("sede")
    .agg(
        ventas_totales=("monto", "sum"),
        ticket_promedio=("monto", "mean"),
        numero_transacciones=("monto", "count")
    )
    .round(2)
    .sort_values("ventas_totales", ascending=False)
)

resumen_sede

,ventas_totales,ticket_promedio,numero_transacciones
sede,,,
Balbuena,19800,4950.0,4
Nezahualcoyotl,12600,2520.0,5
Chalco,6200,1550.0,4
Coapa,6100,1525.0,4


## MARCAR RIESGO

In [6]:
ventas_promedio = resumen_sede["ventas_totales"].mean()
ticket_promedio_global = resumen_sede["ticket_promedio"].mean()

resumen_sede["riesgo"] = (
    (resumen_sede["ventas_totales"] < ventas_promedio) &
    (resumen_sede["ticket_promedio"] < ticket_promedio_global)
)

resumen_sede

,ventas_totales,ticket_promedio,numero_transacciones,riesgo
sede,,,,
Balbuena,19800,4950.0,4,False
Nezahualcoyotl,12600,2520.0,5,False
Chalco,6200,1550.0,4,True
Coapa,6100,1525.0,4,True


## Hallazgo 3: Identificación de sedes en riesgo

El análisis permite identificar sedes con ventas totales y ticket promedio por debajo del promedio general, lo que representa un posible riesgo comercial si no se toman acciones correctivas.

## CLASIFICAR

In [7]:
def clasificar_sede(row):
    if row["ventas_totales"] >= ventas_promedio and row["ticket_promedio"] >= ticket_promedio_global:
        return "Alto desempeño"
    elif row["ventas_totales"] >= ventas_promedio:
        return "Buen volumen, bajo ticket"
    elif row["ticket_promedio"] >= ticket_promedio_global:
        return "Buen ticket, bajo volumen"
    else:
        return "Riesgo comercial"

resumen_sede["clasificacion"] = resumen_sede.apply(clasificar_sede, axis=1)

resumen_sede

,ventas_totales,ticket_promedio,numero_transacciones,riesgo,clasificacion
sede,,,,,
Balbuena,19800,4950.0,4,False,Alto desempeño
Nezahualcoyotl,12600,2520.0,5,False,"Buen volumen, bajo ticket"
Chalco,6200,1550.0,4,True,Riesgo comercial
Coapa,6100,1525.0,4,True,Riesgo comercial


## Interpretación ejecutiva

Las sedes pueden agruparse en cuatro categorías: alto desempeño, buen volumen con bajo ticket, buen ticket con bajo volumen y sedes en riesgo comercial. Esta segmentación permite definir estrategias diferenciadas por sede.

## Recomendaciones

- Priorizar inversión comercial y operativa en sedes de alto desempeño para maximizar retorno.
- Implementar estrategias de upselling en sedes con buen volumen pero bajo ticket promedio.
- Incrementar captación y visibilidad en sedes con buen ticket pero bajo volumen.
- Revisar procesos comerciales y operativos en sedes clasificadas como riesgo comercial.

## Conclusión

El análisis evidencia diferencias claras en desempeño entre sedes. La información obtenida permite tomar decisiones informadas orientadas a crecimiento, optimización y mitigación de riesgo comercial.